In [1]:
ls

data.yaml  labels/       output.txt  trash/     yolo_generate.ipynb
images/    labels_json/  runs/       txt_file/  yolov8n.pt


In [3]:
import json
from glob import glob
from tqdm import tqdm
import os 

#### yolo 학습에 사용될 BBox 및 Class 정의, txt 추출

In [4]:
for i in tqdm(range(1, 66464)):
    num = str(i).zfill(5)
    file_path = f'./labels_json/obj_img_{num}.json'
    
    if not os.path.exists(file_path):
        continue

    with open(file_path) as json_file:
        json_data = json.load(json_file)
        
    file_name = json_data.get('FileName').replace('.jpg', '')
    size = json_data.get('BoundingBox')
    
    # BoundingBox 정보를 좌표 및 크기로 변환하는 함수 정의
    def info(label):
        bounding_box = size.get(label)
        if bounding_box is not None:
            x = ((float(bounding_box[0]) + (float(bounding_box[2]) - float(bounding_box[0])) / 2)/json_data.get('size')[0])
            y = ((float(bounding_box[1]) + (float(bounding_box[3]) - float(bounding_box[1])) / 2)/json_data.get('size')[1])
            w = ((float(bounding_box[2]) - float(bounding_box[0]))/json_data.get('size')[0])
            h = ((float(bounding_box[3]) - float(bounding_box[1]))/json_data.get('size')[1])
            return x, y, w, h
        else:
            return None
        
    # 눈과 핸드폰의 BoundingBox 정보 추출
    leye_info = info('Leye')
    reye_info = info('Reye')
    phone_info = info('Phone')
    
    # 텍스트 파일에 BoundingBox 정보 작성        
    with open(f'./labels/{file_name}.txt', 'a') as text_file:
        if leye_info is not None:
            text_file.write(f"0 {' '.join(map(str, leye_info))}\n")
        if reye_info is not None:
            text_file.write(f"1 {' '.join(map(str, reye_info))}\n")
        if phone_info is not None:
            text_file.write(f"2 {' '.join(map(str, phone_info))}\n")

100%|██████████| 66463/66463 [04:30<00:00, 245.29it/s] 


In [7]:
path = './labels_json'
lst = os.listdir(path)
img_lst = []

for i in tqdm(range(len(lst))):
    filename = lst[i]
    num = filename.split('_')[2].split('.')[0]
    img_lst.append(num)

100%|██████████| 66358/66358 [00:00<00:00, 1124234.56it/s]


#### train_test_split 진행

In [8]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(img_lst, test_size = 0.2)

In [9]:
dir = '/home/work/VisionAI/subeen/images/'
for i in train:
    with open(f'./txt_file/train.txt','a') as text_file:
        text_file.write(f'{dir}obj_img_{i}.jpg\n')

In [10]:
dir = '/home/work/VisionAI/subeen/images/'
for i in test:
    with open(f'./txt_file/test.txt','a') as text_file:
        text_file.write(f'{dir}obj_img_{i}.jpg\n')

In [11]:
from sklearn.model_selection import train_test_split
train1, val = train_test_split(train, test_size = 0.2)

In [12]:
dir = '/home/work/VisionAI/subeen/images/'
for i in train1:
    with open(f'./txt_file/train1.txt','a') as text_file:
        text_file.write(f'{dir}obj_img_{i}.jpg\n')

In [13]:
dir = '/home/work/VisionAI/subeen/images/'
for i in val:
    with open(f'./txt_file/val.txt','a') as text_file:
        text_file.write(f'{dir}obj_img_{i}.jpg\n')